In [123]:
import pandas as pd
from pymongo import MongoClient, GEO2D
from pandas.io.json import json_normalize

In [124]:
#madrid
nearLocation = {
    "lat": 40.395093,
    "long": -3.689154  
}

location=[-3.689154, 40.395093]

In [125]:
# Obtener datos de mongo con el indice 2d geo.
def get_geo_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.offices
    datos_mongo = db.offices
    cursor_geo= datos_mongo.find()
    df = pd.DataFrame(cursor_geo)
    df = df.drop('_id', axis=1)
    return df

In [126]:
def get_offices_near(longitud, latitud):
    df_location = pd.DataFrame(datos_mongo.find({
                                "2Dgeo": {
                                 "$near": {
                                   "$geometry": {
                                      "type": "Point" ,
                                      "coordinates": [ longitud, latitud ]
                                   },
                                   "$maxDistance": 2000, # In meters
                                 }
                               }
                            }))
    return df_location.drop('_id', axis=1)
    

In [130]:
#Main
df_geo_mongo = get_geo_data()
lat = float(df_geo_mongo[0:1].latitude)
long = float(df_geo_mongo[0:1].longitude)
df_location= get_offices_near(long, lat)

In [131]:
df_location

'''
puntos:
video juegos - 100
software - 50
big company - 25
startup - 50
'''

,2Dgeo,category_code,city,country_code,founded_year,latitude,longitude,name,number_of_employees,type
0,"[-122.4013217, 37.7814765]",software,San Francisco,USA,2004,37.781476,-122.401322,UCWeb,1700,BigCompany
1,"[-122.396744, 37.780134]",games_video,San Francisco,USA,2008,37.780134,-122.396744,Scavenja,8,Startup
2,"[-122.407709, 37.781754]",software,San Francisco,USA,2007,37.781754,-122.407709,PX Interactive,8,BigCompany
3,"[-122.407709, 37.781754]",software,San Francisco,USA,2007,37.781754,-122.407709,PX Interactive,8,BigCompany
4,"[-122.401245, 37.786942]",games_video,San Francisco,USA,2006,37.786942,-122.401245,Kongregate,20,BigCompany
5,"[-122.399972, 37.787092]",games_video,San Francisco,USA,2006,37.787092,-122.399972,Curse,58,BigCompany
6,"[-122.395785, 37.77772]",games_video,San Francisco,USA,2006,37.777720,-122.395785,GotGame,15,BigCompany
7,"[-122.393913, 37.780716]",games_video,San Francisco,USA,2006,37.780716,-122.393913,CastTV,0,BigCompany
8,"[-122.402759, 37.787646]",software,San Francisco,USA,2006,37.787646,-122.402759,Keibi Technologies,30,BigCompany
9,"[-122.393229, 37.781265]",games_video,San Francisco,USA,2008,37.781265,-122.393229,Crunchyroll,50,BigCompany
